In [1]:
import pandas as pd

import numpy as np

import csv,sqlite3

In [2]:
# conn = sqlite3.connect("predictions_sql.db")
# cursor = conn.cursor()

In [52]:
df = pd.read_csv("predictions.csv", header=None, names=["user_id", "item_id", "rating_5"])

In [4]:
# df.to_sql("predictions_tb", conn, index=False)

In [53]:
# read from sql db
# pred_df = pd.read_sql_query("select * from predictions_tb limit 5;", conn)

df.head()

,user_id,item_id,rating_5
0,A2DJV5P2NTWEK0,0001981692,4.191430
1,A2DJV5P2NTWEK0,0002008211,4.699676
2,A2DJV5P2NTWEK0,0002254565,4.702403
3,A2DJV5P2NTWEK0,0002552493,4.645525
4,A2DJV5P2NTWEK0,0002710188,4.791867


In [97]:
# store pickle files
metadata = pd.read_pickle("refined_metadata.pkl")

metadata1 = metadata[['asin', 'title', 'imUrl', 'authors', 'lexile']]

metadata1['lexile'] = pd.to_numeric(np.where(metadata1['lexile'].str[0].str.isalpha()\
                                             , metadata1['lexile'].str[2:-1], metadata1['lexile'].str[:-1]))

metadata1.head()

/home/apoorv/anaconda3/envs/cs670/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,asin,title,imUrl,authors,lexile
0,0001006657,I Want My Dinner,http://ecx.images-amazon.com/images/I/510AQCZV...,"[{""name"":""Tony Ross"",""url"":""/content/scholasti...",240
1,0001981692,Pirates,http://ecx.images-amazon.com/images/I/51WN7BJM...,"[{""name"":""Jim Brew"",""url"":""/content/scholastic...",770
2,0002008211,Haunted,http://ecx.images-amazon.com/images/I/516OPgTS...,"[{""name"":""R. L. Stine"",""url"":""/content/scholas...",460
3,0002199203,Rhino,http://ecx.images-amazon.com/images/I/5189PrGP...,"[{""name"":""Sheila Solomon Klass"",""url"":""/conten...",660
4,0002254565,Footsteps,http://ecx.images-amazon.com/images/I/51NVFMSP...,"[{""name"":""Pamela Rushby"",""url"":""/content/schol...",260


In [7]:
# cursor.execute('''DROP TABLE IF EXISTS metadata_tbl''')
# conn.commit()

# metadata1.to_sql("metadata_tbl", conn, index=False)

In [16]:
# store utility file
utility = pd.read_pickle("utility_df.pkl")

In [17]:
utility1 = utility.stack().to_frame("rating_5")

utility1.reset_index(level=1, inplace=True)
utility1.reset_index(level=0, inplace=True)

utility1.head(10)

,user,item,rating_5
0,A03208803KA5HCUYS17Z1,0062117378,5.0
1,A03208803KA5HCUYS17Z1,0062235656,5.0
2,A03208803KA5HCUYS17Z1,014241543X,5.0
3,A03208803KA5HCUYS17Z1,0142419400,4.0
4,A03208803KA5HCUYS17Z1,0374351384,3.0
5,A03208803KA5HCUYS17Z1,0375866566,2.0
6,A100YDQ9DIP06P,0060589507,5.0
7,A100YDQ9DIP06P,0061130990,3.0
8,A100YDQ9DIP06P,0061730831,5.0
9,A100YDQ9DIP06P,0061921033,5.0


In [10]:
# cursor.execute('''DROP TABLE IF EXISTS utility_tbl''')
# conn.commit()

# utility1.to_sql("utility_tbl", conn, index=False)

In [55]:
# get user lexile rating
metadata2 = metadata1[["asin","lexile"]]

In [56]:
usr_lxl = utility1.merge(metadata2, how='left', left_on='item', right_on='asin')
usr_lxl.head(10)

,user,item,rating_5,asin,lexile
0,A03208803KA5HCUYS17Z1,0062117378,5.0,0062117378,610
1,A03208803KA5HCUYS17Z1,0062235656,5.0,0062235656,580
2,A03208803KA5HCUYS17Z1,014241543X,5.0,014241543X,830
3,A03208803KA5HCUYS17Z1,0142419400,4.0,0142419400,580
4,A03208803KA5HCUYS17Z1,0374351384,3.0,0374351384,720
5,A03208803KA5HCUYS17Z1,0375866566,2.0,0375866566,760
6,A100YDQ9DIP06P,0060589507,5.0,0060589507,150
7,A100YDQ9DIP06P,0061130990,3.0,0061130990,760
8,A100YDQ9DIP06P,0061730831,5.0,0061730831,490
9,A100YDQ9DIP06P,0061921033,5.0,0061921033,720


In [62]:
# We will require timestamp, otherwise hard to defend.
# sort usr_lxl at user, rating level & select first row for each user
usr_lxl2 = usr_lxl.sort_values(by=['user', 'rating_5','lexile'], ascending=[True,False,False]).groupby('user').first().reset_index()
usr_lxl3 = usr_lxl2[["user","lexile"]].rename(columns={'lexile':'lexile_user'})

usr_lxl3.head()

,user,lexile_user
0,A03208803KA5HCUYS17Z1,830
1,A100YDQ9DIP06P,970
2,A100ZQDV7L8PVV,1250
3,A104BOQOO5T2SU,910
4,A104SG9QZ92UAH,1020


In [65]:
# merge user lexile ratings with predictions data & join lexile ratings of books
df2 = df.merge(usr_lxl3, how='left', left_on='user_id', right_on='user').drop('user',axis=1)\
        .merge(metadata1, how='left', left_on='item_id', right_on='asin')

df2.head()

,user_id,item_id,rating_5,lexile_user,asin,title,imUrl,authors,lexile
0,A2DJV5P2NTWEK0,0001981692,4.191430,620,0001981692,Pirates,http://ecx.images-amazon.com/images/I/51WN7BJM...,"[{""name"":""Jim Brew"",""url"":""/content/scholastic...",770
1,A2DJV5P2NTWEK0,0002008211,4.699676,620,0002008211,Haunted,http://ecx.images-amazon.com/images/I/516OPgTS...,"[{""name"":""R. L. Stine"",""url"":""/content/scholas...",460
2,A2DJV5P2NTWEK0,0002254565,4.702403,620,0002254565,Footsteps,http://ecx.images-amazon.com/images/I/51NVFMSP...,"[{""name"":""Pamela Rushby"",""url"":""/content/schol...",260
3,A2DJV5P2NTWEK0,0002552493,4.645525,620,0002552493,Hollywood Dogs,http://ecx.images-amazon.com/images/I/41i7XNGU...,"[{""name"":""Meish Goldish"",""url"":""/content/schol...",720
4,A2DJV5P2NTWEK0,0002710188,4.791867,620,0002710188,Home Run,http://ecx.images-amazon.com/images/I/91DoEE3K...,"[{""name"":""Sara Hoskinson Frommer"",""url"":""/cont...",180


In [66]:
usr_itm_lxl = df2
usr_itm_lxl["lexile_diff"] = abs(usr_itm_lxl["lexile_user"] - usr_itm_lxl["lexile"])

In [67]:
usr_itm_lxl2 = usr_itm_lxl.drop(['asin'],axis=1).sort_values(by=['user_id', 'lexile_diff'], ascending=[False, True])

In [68]:
# total num of users
usr_itm_lxl2.user_id.nunique()

3822

In [17]:
# usr_itm_lxl3 = usr_itm_lxl2.groupby('user_id').head(200)

In [69]:
usr_itm_lxl3 = usr_itm_lxl2

In [70]:
usr_itm_lxl3.head()

,user_id,item_id,rating_5,lexile_user,title,imUrl,authors,lexile,lexile_diff
2389317,AZXNB4Q5QCO2P,0060193263,4.058366,1050,Cowboy,http://ecx.images-amazon.com/images/I/51X1ZSJ4...,"[{""name"":""Robert Klausmeier"",""url"":""/content/s...",1050,0
2389627,AZXNB4Q5QCO2P,0061240273,3.918141,1050,Belong to Me,http://ecx.images-amazon.com/images/I/51iLPqq7...,"[{""name"":""Marisa De los Santos"",""url"":""/conten...",1050,0
2390561,AZXNB4Q5QCO2P,030726579X,3.994089,1050,Heart and Soul,http://ecx.images-amazon.com/images/I/71dHTFmJ...,"[{""name"":""Kadir Nelson"",""url"":""/content/schola...",1050,0
2390691,AZXNB4Q5QCO2P,031259612X,3.929125,1050,Wereling,http://ecx.images-amazon.com/images/I/41oRag1B...,"[{""name"":""Steve Feasey"",""url"":""/content/schola...",1050,0
2390741,AZXNB4Q5QCO2P,0316005363,4.074077,1050,Matter,http://ecx.images-amazon.com/images/I/41TE5qmy...,"[{""name"":""Christopher Cooper"",""url"":""/content/...",1050,0


In [20]:
# cursor.execute('''DROP TABLE IF EXISTS usr_itm_lxl3_tbl''')
# conn.commit()

# usr_itm_lxl2.to_sql("usr_itm_lxl3_tbl", conn, index=False)

In [21]:
# def filters(result, lxl_limit, rat_limit):
#     if len(result)<10:
#         cond = (usr_rows.lexile_diff <= lxl_limit) & (usr_rows.rating_5 >= rat_limit)
#         rows = usr_rows.loc[cond,:]
#         rows = rows.sort_values(by='rating_5', ascending=False)
#         result = result.append(rows, ignore_index=True)
#         usr_rows.drop(rows.index, inplace=True)
#     return result

In [71]:
def filters(result, lxl_limit, rat_limit):
    cond = (usr_itm_lxl3.lexile_diff <= lxl_limit) & (usr_itm_lxl3.rating_5 >= rat_limit)
    rows = usr_itm_lxl3.loc[cond,:]
    rows = rows.sort_values(by=['user_id', 'rating_5'], ascending=[False, False])
    result = result.append(rows, ignore_index=True)
    usr_itm_lxl3.drop(rows.index, inplace=True)
    return result

In [72]:
import gc
gc.collect()

560

In [73]:
# usr_rows = usr_itm_lxl3[usr_itm_lxl3['user_id']==t_user]

result = pd.DataFrame(columns = usr_itm_lxl3.columns)

result = filters(result,50,4.5)     # reco books with lexile_diff < 50 and rating >= 4.5
result = filters(result,50,4.0)
# result = filters(result,100,4.5)
# result = filters(result,100,4.0)

# result = result.head(10)

In [74]:
result.user_id.nunique(), len(result)

(3753, 1975454)

In [75]:
result.to_pickle(path='result_users.pkl')

In [76]:
result = result.groupby('user_id').head(10).reset_index(drop=True)

In [77]:
result.user_id.nunique(), len(result)

(3753, 37102)

In [78]:
result.to_pickle(path='result_users_f.pkl')

In [79]:
result = pd.read_pickle("result_users_f.pkl")

In [81]:
# select 10 users
users = result['user_id'].drop_duplicates().head(10).to_frame().sort_values(by='user_id').reset_index(drop=True)
users

,user_id
0,AZ9CUMDHTWICA
1,AZA55FJD7CCZN
2,AZETMGFX0UZFQ
3,AZFX7FLEHCWXM
4,AZKGCE3LNX7WB
5,AZNZ4V0YQDK6P
6,AZRWNAHNUAQLV
7,AZSV99SDJC242
8,AZWC9XAY34IPW
9,AZXNB4Q5QCO2P


In [100]:
# Books recommended
selected_rec = result.merge(users,on='user_id').sort_values(by='user_id',ascending=True).reset_index(drop=True)
print len(selected_rec)

selected_rec.to_pickle('selected_rec.pkl')
selected_rec.head()

100


,user_id,item_id,rating_5,lexile_user,title,imUrl,authors,lexile,lexile_diff
0,AZ9CUMDHTWICA,1402238355,4.922659,700,More Bears!,http://ecx.images-amazon.com/images/I/51gxyK5Z...,"[{""name"":""Troy Cummings"",""url"":""/content/schol...",740,40
1,AZ9CUMDHTWICA,0312384289,4.981751,700,Alabama Moon,http://ecx.images-amazon.com/images/I/41ctJ9rM...,"[{""name"":""Watt Key"",""url"":""/content/scholastic...",720,20
2,AZ9CUMDHTWICA,0061582506,4.959716,700,Warriors Super Edition: Bluestar's Prophecy,http://ecx.images-amazon.com/images/I/51omd4FA...,"[{""name"":""Erin Hunter"",""url"":""/content/scholas...",670,30
3,AZ9CUMDHTWICA,0330512765,4.944469,700,The Princess and the Pig,http://ecx.images-amazon.com/images/I/91tdyDqP...,"[{""name"":""Jonathan Emmett"",""url"":""/content/sch...",700,0
4,AZ9CUMDHTWICA,0803738382,4.942508,700,The Mouse with the Question Mark Tail,http://ecx.images-amazon.com/images/I/51CVR4kF...,"[{""name"":""Richard Peck"",""url"":""/content/schola...",680,20


In [84]:
utility2 = utility1.merge(users, left_on='user', right_on='user_id').drop('user_id',axis=1)
utility2.head() # for 10 users

,user,item,rating_5
0,AZ9CUMDHTWICA,0395866189,5.0
1,AZ9CUMDHTWICA,043992376X,5.0
2,AZ9CUMDHTWICA,0618494588,5.0
3,AZ9CUMDHTWICA,0689825501,5.0
4,AZ9CUMDHTWICA,0811818071,5.0


In [101]:
# Books rated by the users
selected_rated = utility2.merge(metadata1,left_on='item', right_on='asin').sort_values(by=['user', 'lexile'],ascending=[True,False])\
.drop('asin',axis=1).reset_index(drop=True)

print len(selected_rated)

selected_rated.to_pickle('selected_rated.pkl')
selected_rated.head()

95


,user,item,rating_5,title,imUrl,authors,lexile
0,AZ9CUMDHTWICA,043992376X,5.0,Snowmen at Christmas,http://ecx.images-amazon.com/images/I/511lE8g0...,"[{""name"":""Mark Buehner"",""url"":""/content/schola...",700
1,AZ9CUMDHTWICA,0811818071,5.0,"Olive, the Other Reindeer",http://ecx.images-amazon.com/images/I/51yLGHyZ...,"[{""name"":""Vivian Walsh"",""url"":""/content/schola...",590
2,AZ9CUMDHTWICA,0689825501,5.0,Kate and the Beanstalk,http://ecx.images-amazon.com/images/I/51mN0YIY...,"[{""name"":""Giselle Potter"",""url"":""/content/scho...",440
3,AZ9CUMDHTWICA,0395866189,5.0,Piggie Pie!,http://ecx.images-amazon.com/images/I/51NJRkTo...,"[{""name"":""Margie Palatini"",""url"":""/content/sch...",420
4,AZ9CUMDHTWICA,0618494588,5.0,Our Library,http://ecx.images-amazon.com/images/I/516CTlOj...,"[{""name"":""Maggie Smith"",""url"":""/content/schola...",390
